In [41]:
### Transfer Learning 
import tensorflow as tf
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dropout,Input,Flatten,Dense,MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
tf.test.is_gpu_available()

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


False

In [42]:
batchsize=8

In [37]:
train_datagen = ImageDataGenerator(rescale=1./255, rotation_range=0.2, shear_range=0.2,
zoom_range=0.2, width_shift_range=0.2, height_shift_range=0.2, validation_split=0.2)

In [43]:
train_datagen= ImageDataGenerator(rescale=1./255, rotation_range=0.2,shear_range=0.2,
    zoom_range=0.2,width_shift_range=0.2,
    height_shift_range=0.2, validation_split=0.2)

train_data= train_datagen.flow_from_directory(r'C:\Users\sharo\projectsXD\ProjectsLetsFuckitUp\Drowsiness Detection\prepared_data\train',
                                target_size=(80,80),batch_size=batchsize,class_mode='categorical',subset='training' )

validation_data= train_datagen.flow_from_directory(r'C:\Users\sharo\projectsXD\ProjectsLetsFuckitUp\Drowsiness Detection\prepared_data\train',
                                target_size=(80,80),batch_size=batchsize,class_mode='categorical', subset='validation')



Found 19171 images belonging to 2 classes.
Found 4792 images belonging to 2 classes.


In [44]:
test_datagen = ImageDataGenerator(rescale=1./255)

In [46]:
test_data = test_datagen.flow_from_directory(r'C:\Users\sharo\projectsXD\ProjectsLetsFuckitUp\Drowsiness Detection\prepared_data\test',
                                target_size=(80,80),batch_size=batchsize,class_mode='categorical')




Found 6500 images belonging to 2 classes.


In [47]:
bmodel = InceptionV3(include_top=False, weights='imagenet', input_tensor=Input(shape=(80,80,3)))
hmodel = bmodel.output
hmodel = Flatten()(hmodel)
hmodel = Dense(64, activation='relu')(hmodel)
hmodel = Dropout(0.5)(hmodel)
hmodel = Dense(2,activation= 'softmax')(hmodel)

model = Model(inputs=bmodel.input, outputs= hmodel)
for layer in bmodel.layers:
    layer.trainable = False

In [24]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(8, 80, 80, 3)]     0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (8, 39, 39, 32)      864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (8, 39, 39, 32)      96          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (8, 39, 39, 32)      0           batch_normalization[0][0]        
______________________________________________________________________________________________

In [48]:

from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping, ReduceLROnPlateau

In [56]:
checkpoint = ModelCheckpoint(r'C:\Users\sharo\projectsXD\ProjectsLetsFuckitUp\Drowsiness Detection\models\models.h5',
                            monitor='val_loss',save_best_only=True,verbose=3)

earlystop = EarlyStopping(monitor = 'val_loss', patience=7, verbose= 3, restore_best_weights=True)

learning_rate = ReduceLROnPlateau(monitor= 'val_loss', patience=3, verbose= 3, )

callbacks=[checkpoint,earlystop,learning_rate]


In [57]:
model.compile(optimizer='Adam', loss='categorical_crossentropy',metrics=['accuracy'])

model.fit_generator(train_data,steps_per_epoch=train_data.samples//batchsize,
                   validation_data=validation_data,
                   validation_steps=validation_data.samples//batchsize,
                   callbacks=callbacks,
                    epochs=50)

c:\Users\sharo\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:1940: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/50
2396/2396 [==============================] - 430s 173ms/step - loss: 0.0979 - accuracy: 0.9676 - val_loss: 0.0394 - val_accuracy: 0.9904

Epoch 00001: val_loss improved from inf to 0.03939, saving model to C:\Users\sharo\projectsXD\ProjectsLetsFuckitUp\Drowsiness Detection\models\models.h5
Epoch 2/50
2396/2396 [==============================] - 309s 129ms/step - loss: 0.0948 - accuracy: 0.9673 - val_loss: 0.0494 - val_accuracy: 0.9860

Epoch 00002: val_loss did not improve from 0.03939
Epoch 3/50
2396/2396 [==============================] - 303s 126ms/step - loss: 0.0952 - accuracy: 0.9667 - val_loss: 0.0348 - val_accuracy: 0.9902

Epoch 00003: val_loss improved from 0.03939 to 0.03480, saving model to C:\Users\sharo\projectsXD\ProjectsLetsFuckitUp\Drowsiness Detection\models\models.h5
Epoch 4/50
2396/2396 [==============================] - 293s 122ms/step - loss: 0.0916 - accuracy: 0.9688 - val_loss: 0.0356 - val_accuracy: 0.9917

Epoch 00004: val_loss did not improve from 

In [58]:
# MODEL EVALUATION
acc_tr, loss_tr = model.evaluate_generator(train_data)
print(acc_tr)
print(loss_tr)

c:\Users\sharo\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:1973: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  warnings.warn('`Model.evaluate_generator` is deprecated and '


0.07763838768005371
0.9715194702148438


In [54]:
acc_tr, loss_tr = model.evaluate_generator(train_data)
print(acc_tr)
print(loss_tr)

0.0752595067024231
0.9726148843765259


In [59]:
acc_tr, loss_tr = model.evaluate_generator(train_data)
print(acc_tr)
print(loss_tr)

0.07587528973817825
0.9732930064201355
